In [19]:
import requests
import json, re, time
from datetime import datetime
import pandas as pd

In [20]:
with open('api.key','r') as f:
    APIKEY = f.read()

HEADERS = {'apikey': APIKEY,
            'accept': 'application/json',
            'content-type': 'application/json',
            'cookie': 'ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip; ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip'
          }

API_ENDPOINT = 'https://kong.speedcheckerapi.com:8443/ProbeAPIv2/'

probeInfoProperties = [
        "ASN",
        "CityName",
        "ConnectionType",
        "CountryCode",
        "DNSResolver",
        "GeolocationAccuracy",
        "IPAddress",
        "Latitude",
        "Longitude",
        "Network",
        "NetworkID",
        "Platform",
        "ProbeID",
        "Version"
    ]

In [21]:
def getTestSettings(cc, url):
    
    destinations = [url]
    json_test = {
                  "testSettings": {
                    "TestCount": 10,
                    "Sources": [
                      {
                        "CountryCode": cc,
                        "Platform": "Android"
                      }
                    ],
                    "Destinations": destinations,
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

In [22]:
def getTestSettingsEUUS(url):
    
    destinations = [url]
    json_test = {
                  "testSettings": {
                    "TestCount": 2,
                    "Sources": [
                      {
                        "CountryCode": "US"
                      },
                      {
                        "CountryCode": "FR"
                      },
                      {
                        "CountryCode": "DE"
                      },
                      {
                        "CountryCode": "NL"
                      },
                      {
                        "CountryCode": "GB"
                      }  
                    ],
                    "Destinations": destinations,
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

### StartPageLoadTest function

In [23]:
def startPageLoadTest(test_settings):
    
    test_url = API_ENDPOINT + "StartPageLoadTest"
    try:
        r = requests.post(test_url, data=json.dumps(test_settings), headers=HEADERS)
    except requests.exceptions.RequestException as e:
        return "Request FAILED"
    
    res = json.loads(r.text)
        
    if ("OK" == res['StartPageLoadTestResult']['Status']['StatusText']):
        return res['StartPageLoadTestResult']['TestID']
    else:
        return "FAILED"

### retrievePageLoadTestResults function

In [24]:
def retrievePageLoadTestResults(testID):
    
    url = API_ENDPOINT + "GetPageLoadResults?apikey=" + APIKEY + "&testID=" + testID
    
    try:    
        r = requests.get(url, headers=HEADERS)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        return []
    
    res = json.loads(r.text)
    return res['PageLoadTestResults']

## Retrieve results

In [25]:
def getResultsFromTestIDs(df_tests_url):
    df_results = pd.DataFrame(columns=['domain',
                                       'cc',
                                       'ProbeInfo',
                                       'TestDateTime',
                                       'StatusCode',
                                       'StatusText',
                                       'DNSLookupTime', 
                                       'Destination', 
                                       'HTTPStatus', 
                                       'InitialConnection', 
                                       'NumberOfRequests', 
                                       'PageLoadTime', 
                                       'SSLNegotiationTime', 
                                       'TTFB', 
                                       'TotalDownloadedBytes'
                                      ])
    
    #Exclude FAILED items
    for index, row in df_tests_url.loc[df_tests_url.testID!='FAILED'].iterrows():
        
        domain = row['domain']
        cc = row['cc']
        testID = row['testID']
        pageLoadTestResults =  retrievePageLoadTestResults(testID)

        if (pageLoadTestResults is None or len(pageLoadTestResults) < 1):
            continue
        
        #print(pageLoadTestResults)
        #print("€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€€")
        
        statusCode = ""
        statusText = ""
        dnsLookupTime = ""
        destination = ""
        hTTPStatus = ""
        initialConnection = ""
        numberOfRequests = ""
        pageLoadTime = ""
        sslNegotiationTime = ""
        ttfb = ""
        totalDownloadedBytes = ""
        
        for res in pageLoadTestResults:
        
            probeInfo = res['ProbeInfo']
            testDateTime = res['TestDateTime']

            #extract the epoch
            m = re.search('/Date\((\d+)\+0000\)/', testDateTime)
            if m:
                testDateTime = m.group(1)[:-3]

            #convert into human readable format
            testDateTime = datetime.fromtimestamp(int(testDateTime)).strftime('%Y-%m-%d')
            try:
                statusCode = res['TestStatus']['StatusCode']
                statusText = res['TestStatus']['StatusText']
                dnsLookupTime = res['DNSLookupTime']
                destination =  res['Destination']
                hTTPStatus = res['HTTPStatus']
                initialConnection = res['InitialConnection']
                numberOfRequests =  res['NumberOfRequests'] 
                pageLoadTime =  res['PageLoadTime'] 
                sslNegotiationTime = res['SSLNegotiationTime'] 
                ttfb = res['TTFB']
                totalDownloadedBytes = res['TotalDownloadedBytes']
            except:
                pass

            df_results = df_results.append({
                'domain': domain,
                'cc': cc,
                'ProbeInfo':probeInfo,
                'TestDateTime': testDateTime,
                'StatusCode':statusCode,
                'StatusText':statusText,
                'DNSLookupTime':dnsLookupTime,
                'Destination':destination,
                'HTTPStatus':hTTPStatus,
                'InitialConnection':initialConnection,
                'NumberOfRequests':numberOfRequests,
                'PageLoadTime':pageLoadTime,
                'SSLNegotiationTime':sslNegotiationTime,
                'TTFB':ttfb,
                'TotalDownloadedBytes':totalDownloadedBytes
            }, ignore_index=True)

    return df_results

In [26]:
def runMeasurements(df_links, url_type, filename):
    df_testID = pd.DataFrame(columns=['domain', 'cc', 'testID'])

    for index, row in df_links.iterrows():
        domain = row['domain']
        cc = row['cc']
        url = row[url_type]

        test_settings = getTestSettings(cc, url)
        testID = startPageLoadTest(test_settings)
        print(testID)
        
        df_testID = df_testID.append({'domain': domain, 'cc': cc, 'testID': testID}, ignore_index=True)
    
    df_testID.to_csv(filename, sep='|', encoding='utf-8', index=False)    
    

In [27]:
def runMeasurementsEUUS(df_links, url_type, filename):
    df_testID = pd.DataFrame(columns=['domain', 'cc', 'testID'])

    for index, row in df_links.iterrows():
        domain = row['domain']
        cc = row['cc']
        url = row[url_type]

        test_settings = getTestSettingsEUUS(url)
        testID = startPageLoadTest(test_settings)
        print(testID)
        
        df_testID = df_testID.append({'domain': domain, 'cc': cc, 'testID': testID}, ignore_index=True)
    
    df_testID.to_csv(filename, sep='|', encoding='utf-8', index=False)    

In [40]:
df_links = pd.read_csv('data/links_sample.csv', sep='|', encoding='utf-8', keep_default_na=False)

In [41]:
#runMeasurements(df_links, 'url', 'data/url_testID.csv')
#runMeasurements(df_links, 'ori_amp_url', 'data/ori_amp_url_testID.csv')
runMeasurements(df_links, 'amp_viewer_url', 'data/amp_viewer_url_testID-23072018.csv')
#runMeasurements(df_links, 'amp_cdn_url', 'data/amp_cdn_url_testID.csv')
#runMeasurementsEUUS(df_links, 'amp_viewer_url', 'data/amp_viewer_url_testID_EUUS-21072018.csv')


8d648d81-4533-4952-a879-966afb72937f
e6145404-f51b-4ba9-8ea2-9415db7a3a5d
bd1fd68c-1099-412a-a6d5-121b7c039965
c007ee37-d6ab-46a3-b495-fc8214363c79
b1a7d16e-ff59-4757-b5cc-aa87bb16e903
339e5bb4-fe54-4517-a96b-d8827e853e6c
93ca4add-a304-4307-b7e5-1ba0fde4229d
4a8bd245-fb9c-4eba-82d9-2812cba19cab
38612f16-f053-4ba7-be57-24c614336258
7dc84bc3-1579-41b4-b276-1be6c4816218
85b29c9f-622b-4a2b-a194-af4133f4d9b3
9dc5b7a1-d753-4e32-be49-8ced6e2db579
bf3ff832-bbd4-4941-bf3b-add2c20d4cfe
c1756a4b-543a-4a53-bee0-3bf2a27cebef
4ce6207e-0919-4138-8d83-a4e0a7a19d84
34aaaaff-6521-4878-bc97-699a0498f472
794a5df6-bd0c-4958-bdfe-2e80af7d8f78
6fa61fec-bc86-4227-8759-ca552bd7e3fd
9d6dd6b4-2755-406c-8fdc-97f2f94e492d
e88ff9b5-d195-4dcd-9ae1-ca5dde3a60d0
1f1ff8e6-3977-4c8c-a360-367daf118fd8
b8a93db0-74d7-4285-b33f-9c1dac7377a0
870664bc-dd59-4232-a7f4-14c41ffbe628
3dc00b1e-fb34-40a8-bf6c-db1027233dfe
7f1f80b6-f329-4cfd-93ab-73f6184717f7
1e2c8100-e0c4-48dd-93dd-9af6ebd42e5b
a2bd1f12-35cc-448d-8125-6b8d73ce7933
9

In [42]:
#df_testIDs_url = pd.read_csv('data/url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_ori_amp = pd.read_csv('data/ori_amp_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
df_testIDs_amp_viewer = pd.read_csv('data/amp_viewer_url_testID-23072018.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_cdn = pd.read_csv('data/amp_cdn_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_viewer_urlEUUS = pd.read_csv('data/amp_viewer_url_testID_EUUS-21072018.csv', sep='|', encoding='utf-8', keep_default_na=False)


In [44]:
#df_results_ori_amp = getResultsFromTestIDs(df_testIDs_ori_amp)
df_results_amp_viewer = getResultsFromTestIDs(df_testIDs_amp_viewer)
#df_results_amp_cdn = getResultsFromTestIDs(df_testIDs_amp_cdn)
#df_results_url = getResultsFromTestIDs(df_testIDs_url)
#df_results_url_EUUS = getResultsFromTestIDs(df_testIDs_urlEUUS)
#df_results_amp_viewer_url_EUUS = getResultsFromTestIDs(df_testIDs_amp_viewer_urlEUUS)


In [46]:
#df_results_url.to_csv("data/url_results" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
#df_results_ori_amp.to_csv("data/ori_amp_url_results.csv", sep='|', encoding='utf-8', index=False)
df_results_amp_viewer.to_csv("data/amp_viewer_url_results-2307201.csv", sep=';', encoding='utf-8', index=False)
#df_results_amp_cdn.to_csv("data/amp_cdn_url_results.csv", sep='|', encoding='utf-8', index=False)
#df_results_url_EUUS.to_csv("data/url_results_EUUS" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
#df_results_amp_viewer_url_EUUS.to_csv("data/amp_viewer_url_results-" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
